Installs for all the needed packages.

This code is loading both the json file contaning all code snippents and there real label. It also loads the csv file that contains all of my collected CVE vulnerability summaries.

In [ ]:
from transformers import cached_path
cached_path.clear_cache()

In [ ]:
!pip3 install -q -U bitsandbytes==0.42.0
!pip3 install -q -U peft==0.8.2
!pip3 install -q -U trl==0.7.10
!pip3 install -q -U accelerate==0.27.1
!pip3 install -q -U datasets==2.17.0
!pip3 install -q -U transformers==4.38.0

In [ ]:
import os
import json
import pandas as pd
import random
import torch
import transformers
import torch
import re
from google.colab import userdata
from datasets import load_dataset
from trl import SFTTrainer
from peft import LoraConfig
from transformers import AutoTokenizer, AutoModelForCausalLM
from transformers import BitsAndBytesConfig, GemmaTokenizer
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

In [ ]:
# Define the path to your JSON file
code_data = "/content/drive/MyDrive/processedData.json"
cve_summaries_path = "/content/drive/MyDrive/training_data.csv"

# This will store the corrected JSON objects
corrected_data = [] # code_data

# Read the file and convert each JSON object correctly
with open(code_data, 'r') as file:
    for line in file:
        try:
            # Try to parse each line as a separate JSON object
            json_object = json.loads(line)
            corrected_data.append(json_object)
        except json.JSONDecodeError:
            print("Error decoding JSON from line:", line)

# Pretty print the first JSON object
print(json.dumps(corrected_data[0], indent=4))

# Load CSV data
df = pd.read_csv(cve_summaries_path)

print("\nCSV Data Sample:")
print(df.head())  # Display the first 5 rows of the DataFrame

{
    "func": "int _gnutls_ciphertext2compressed(gnutls_session_t session,\n\t\t\t\t  opaque * compress_data,\n\t\t\t\t  int compress_size,\n\t\t\t\t  gnutls_datum_t ciphertext, uint8 type)\n{\n    uint8 MAC[MAX_HASH_SIZE];\n    uint16 c_length;\n    uint8 pad;\n    int length;\n    mac_hd_t td;\n    uint16 blocksize;\n    int ret, i, pad_failed = 0;\n    uint8 major, minor;\n    gnutls_protocol_t ver;\n    int hash_size =\n\t_gnutls_hash_get_algo_len(session->security_parameters.\n\t\t\t\t  read_mac_algorithm);\n\n    ver = gnutls_protocol_get_version(session);\n    minor = _gnutls_version_get_minor(ver);\n    major = _gnutls_version_get_major(ver);\n\n    blocksize = _gnutls_cipher_get_block_size(session->security_parameters.\n\t\t\t\t\t      read_bulk_cipher_algorithm);\n\n    /* initialize MAC \n     */\n    td = mac_init(session->security_parameters.read_mac_algorithm,\n\t\t  session->connection_state.read_mac_secret.data,\n\t\t  session->connection_state.read_mac_secret.size, ver

In [ ]:
def get_sample_functions(data, target=1, num_samples=5):
    """ Sample functions based on their vulnerability status. """
    filtered_funcs = [func for func in data if func['target'] == target]
    num_samples = min(len(filtered_funcs), num_samples)  # Ensure we don't sample more than available
    return random.sample(filtered_funcs, num_samples)

In [ ]:
os.environ["HF_TOKEN"] = userdata.get('HF_TOKEN')

Gemma Baseline Testing

In [ ]:
model_id = "google/gemma-2b-it"
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_id, token=os.environ['HF_TOKEN'])
model = AutoModelForCausalLM.from_pretrained(model_id,
                                             quantization_config=bnb_config,
                                             device_map={"":0},
                                             token=os.environ['HF_TOKEN'])

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
def analyze_function(function):
    """ Generate the prompt and analyze the function using the model directly. """
    prompt = f"Analyze this function for security vulnerabilities: \n{function['func']}"
    inputs = tokenizer(prompt, return_tensors="pt", padding=True).to('cuda:0')
    outputs = model.generate(**inputs, max_new_tokens=256, do_sample=True, temperature=0.7, top_k=50, top_p=0.95)
    result = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return clean_and_interpret_response(result)

def clean_and_interpret_response(text):
    """ Interpret the response to determine vulnerability. """
    not_vulnerable_patterns = [
        r'\bis (not|secure|safe)\b', r'\b(no significant security vulnerabilities)\b',
        r'\b(no|without) obvious vulnerabilities\b', r'\btherefore, it is not vulnerable\b', r'\bclean'
    ]
    if any(re.search(pattern, text, re.IGNORECASE) for pattern in not_vulnerable_patterns):
        return 0
    return 1

# Combine samples from clean and vulnerable sets
samples = get_sample_functions(corrected_data, target=0, num_samples=5) + get_sample_functions(corrected_data, target=1, num_samples=5)

# Analyze each sample and store results
predictions = []
true_labels = [func['target'] for func in samples]
for func in samples:
    prediction = analyze_function(func)
    predictions.append(prediction)

# Compute and print performance metrics
accuracy = accuracy_score(true_labels, predictions)
precision = precision_score(true_labels, predictions)
recall = recall_score(true_labels, predictions)
f1 = f1_score(true_labels, predictions)

print(f"Accuracy: {accuracy:.2f}")
print(f"Precision: {precision:.2f}")
print(f"Recall: {recall:.2f}")
print(f"F1 Score: {f1:.2f}")

Accuracy: 0.60
Precision: 0.57
Recall: 0.80
F1 Score: 0.67


llama Baseline Setup

In [ ]:
model_id_llama = "meta-llama/Meta-Llama-3-8B-Instruct"
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

In [ ]:
tokenizer_llama = AutoTokenizer.from_pretrained(model_id_llama, token=os.environ['HF_TOKEN'], force_download=True)
model_llama = AutoModelForCausalLM.from_pretrained(model_id_llama,
                                                   quantization_config=bnb_config,
                                                   device_map={"":0},
                                                   token=os.environ['HF_TOKEN'],
                                                   force_download=True)

tokenizer_config.json:   0%|          | 0.00/50.6k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/50.6k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/73.0 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/50.6k [00:00<?, ?B/s]

...

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/177 [00:00<?, ?B/s]

In [ ]:
def analyze_function_llama(function):
    prompt = f"Analyze this function for security vulnerabilities: \n{function['func']}"
    inputs = tokenizer_llama(prompt, return_tensors="pt", padding=True).to('cuda:0')
    outputs = model_llama.generate(**inputs, max_new_tokens=256, do_sample=True, temperature=0.7, top_k=50, top_p=0.95)
    result = tokenizer_llama.decode(outputs[0], skip_special_tokens=True)
    return clean_and_interpret_response_llama(result)

def clean_and_interpret_response_llama(text):
    """ Interpret the response to determine vulnerability. """
    not_vulnerable_patterns = [
        r'\bis (not|secure|safe)\b', r'\b(no significant security vulnerabilities)\b',
        r'\b(no|without) obvious vulnerabilities\b', r'\btherefore, it is not vulnerable\b', r'\bclean'
    ]
    if any(re.search(pattern, text, re.IGNORECASE) for pattern in not_vulnerable_patterns):
        return 0
    return 1

# Combine samples from clean and vulnerable sets
samples = get_sample_functions(corrected_data, target=0, num_samples=5) + get_sample_functions(corrected_data, target=1, num_samples=5)

# Analyze each sample and store results
predictions = []
true_labels = [func['target'] for func in samples]
for func in samples:
    prediction = analyze_function(func)
    predictions.append(prediction)

# Compute and print performance metrics
accuracy = accuracy_score(true_labels, predictions)
precision = precision_score(true_labels, predictions)
recall = recall_score(true_labels, predictions)
f1 = f1_score(true_labels, predictions)

print(f"Accuracy: {accuracy:.2f}")
print(f"Precision: {precision:.2f}")
print(f"Recall: {recall:.2f}")
print(f"F1 Score: {f1:.2f}")

Accuracy: 0.70
Precision: 1.00
Recall: 0.40
F1 Score: 0.57


PHI-3 Baseline Setup

In [ ]:
model_id_phi3 = "microsoft/Phi-3-mini-128k-instruct"
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

In [ ]:
tokenizer_phi3 = AutoTokenizer.from_pretrained(model_id_phi3,
                                               token=os.environ['HF_TOKEN'],
                                               force_download=True,
                                               trust_remote_code=True)

model_phi3 = AutoModelForCausalLM.from_pretrained(model_id_phi3,
                                                  token=os.environ['HF_TOKEN'],
                                                  force_download=True,
                                                  trust_remote_code=True)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/3.17k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/293 [00:00<?, ?B/s]

...

model.safetensors.index.json:   0%|          | 0.00/16.3k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/2.67G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/172 [00:00<?, ?B/s]

In [ ]:
def analyze_function_phi3(function):
    prompt = f"Analyze this function for security vulnerabilities: \n{function['func']}"
    inputs = tokenizer_phi3(prompt, return_tensors="pt", padding=True).to('cuda:0')
    outputs = model_phi3.generate(**inputs, max_new_tokens=256, do_sample=True, temperature=0.7, top_k=50, top_p=0.95)
    result = tokenizer_phi3.decode(outputs[0], skip_special_tokens=True)
    return clean_and_interpret_response_phi(result)

def clean_and_interpret_response_phi(text):
    """ Interpret the response to determine vulnerability. """
    not_vulnerable_patterns = [
        r'\bis (not|secure|safe)\b', r'\b(no significant security vulnerabilities)\b',
        r'\b(no|without) obvious vulnerabilities\b', r'\btherefore, it is not vulnerable\b', r'\bclean'
    ]
    if any(re.search(pattern, text, re.IGNORECASE) for pattern in not_vulnerable_patterns):
        return 0
    return 1

# Combine samples from clean and vulnerable sets
samples = get_sample_functions(corrected_data, target=0, num_samples=5) + get_sample_functions(corrected_data, target=1, num_samples=5)

# Analyze each sample and store results
predictions = []
true_labels = [func['target'] for func in samples]
for func in samples:
    prediction = analyze_function(func)
    predictions.append(prediction)

# Compute and print performance metrics
accuracy = accuracy_score(true_labels, predictions)
precision = precision_score(true_labels, predictions)
recall = recall_score(true_labels, predictions)
f1 = f1_score(true_labels, predictions)

print(f"Accuracy: {accuracy:.2f}")
print(f"Precision: {precision:.2f}")
print(f"Recall: {recall:.2f}")
print(f"F1 Score: {f1:.2f}")

Accuracy: 0.50
Precision: 0.50
Recall: 0.60
F1 Score: 0.55


Quen2 Baseline Setup

In [ ]:
model_id_qwen2 = "Qwen/Qwen2-7B-Instruct"
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

In [ ]:
tokenizer_qwen2 = AutoTokenizer.from_pretrained(model_id_qwen2, token=os.environ['HF_TOKEN'])
model_qwen2 = AutoModelForCausalLM.from_pretrained(model_id_qwen2,
                                                   quantization_config=bnb_config,
                                                   device_map={"":0},
                                                   token=os.environ['HF_TOKEN'])

tokenizer_config.json:   0%|          | 0.00/1.29k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/7.03M [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


...

model-00002-of-00004.safetensors:   0%|          | 0.00/3.86G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/3.86G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/3.56G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/243 [00:00<?, ?B/s]

In [ ]:
def analyze_function_qwen2(function):
    prompt = f"Analyze this function for security vulnerabilities: \n{function['func']}"
    inputs = tokenizer_qwen2(prompt, return_tensors="pt", padding=True).to('cuda:0')
    outputs = model_qwen2.generate(**inputs, max_new_tokens=256, do_sample=True, temperature=0.7, top_k=50, top_p=0.95)
    result = tokenizer_qwen2.decode(outputs[0], skip_special_tokens=True)
    return clean_and_interpret_response_qwen(result)

def clean_and_interpret_response_qwen(text):
    """ Interpret the response to determine vulnerability. """
    not_vulnerable_patterns = [
        r'\bis (not|secure|safe)\b', r'\b(no significant security vulnerabilities)\b',
        r'\b(no|without) obvious vulnerabilities\b', r'\btherefore, it is not vulnerable\b', r'\bclean'
    ]
    if any(re.search(pattern, text, re.IGNORECASE) for pattern in not_vulnerable_patterns):
        return 0
    return 1  # Assume vulnerable if not clearly stated as clean

# Combine samples from clean and vulnerable sets
samples = get_sample_functions(corrected_data, target=0, num_samples=5) + get_sample_functions(corrected_data, target=1, num_samples=5)

# Analyze each sample and store results
predictions = []
true_labels = [func['target'] for func in samples]
for func in samples:
    prediction = analyze_function(func)
    predictions.append(prediction)

# Compute and print performance metrics
accuracy = accuracy_score(true_labels, predictions)
precision = precision_score(true_labels, predictions)
recall = recall_score(true_labels, predictions)
f1 = f1_score(true_labels, predictions)

print(f"Accuracy: {accuracy:.2f}")
print(f"Precision: {precision:.2f}")
print(f"Recall: {recall:.2f}")
print(f"F1 Score: {f1:.2f}")

Accuracy: 0.50
Precision: 0.50
Recall: 0.40
F1 Score: 0.44
